# Introduction

The [HHL algorithm](https://en.wikipedia.org/wiki/Quantum_algorithm_for_linear_systems_of_equations) underlies many quantum machine learning protocols, but it is a highly nontrivial algorithm with lots of conditions. In this notebook, we implement the algorithm to gain a better understanding of how it works and when it works efficiently. The notebook is derived from the [computational appendix](https://gitlab.com/apozas/bayesian-dl-quantum) of the paper [Bayesian Deep Learning on a Quantum Computer](https://arxiv.org/abs/1806.11463). We restrict our attention to inverting a $2\times 2$ matrix, following Pan *et al*.'s implementation [[1](#1)] of the algorithm.

In [ ]:
import numpy as np
import cirq
from cirq_tools import *
from scipy.optimize import minimize
np.set_printoptions(precision=3, suppress=True)
π = np.pi

# Preparation

We define generic unitary gates u3, u1, cu3, cu1 equivalent to those in qiskit so that we can use the same circuit structure with the qiskit version.

In [ ]:
def u3(t, p, l):
    matrix = np.array(
      [[np.cos(t/2.0), -np.exp(1.0j*l)*np.sin(t/2.0)],
       [np.exp(1.0j*p)*np.sin(t/2.0), np.exp(1.0j*(p+l))*np.cos(t/2.0)]])
    return cirq.SingleQubitMatrixGate(matrix=matrix)

def cu3(t, p, l):
    return cirq.ControlledGate(u3(t, p, l))

def u1(l):
    return u3(0, 0, l)

def cu1(l):
    return cu3(0, 0, l)

# Setting up the problem

We will solve the equation $Ax=b$ with $A = \frac{1}{2}\begin{bmatrix}3 & 1 \\1 & 3 \\ \end{bmatrix}$ and $b =\begin{bmatrix} 1 \\ 0 \\ \end{bmatrix}$. We will encode the $A$ matrix as a Hamiltonian and $b$ in a register. With ancillas, we will need a total of four qubits and one classical register for post-selection. We add extra qubits to create a swap test to compare our result to the ideal state.

In [ ]:
q = cirq.LineQubit.range(6)
hhl = cirq.Circuit()

The vector $b$ can be encoded as $\left|b\right\rangle = \sum_{i=0}^N b_i\left|i\right\rangle = \left|0\right\rangle$, so no explicit state preparation circuit is needed for this case (this will not be true in general).

# Quantum phase estimation

The next step is to encode the eigenvalues of the matrix $A$ in an additional register. This is done via quantum phase estimation of the evolution described by the Hamiltonian $A$ during some time $t_0$, $\exp(i A t_0)$. The protocol has three steps.

First we prepare the ancilla state $\left|\psi_0\right\rangle=\sum_{\tau=0}^{T-1}\left|\tau\right\rangle$. Why this state? It will control the time evolution: it is like a clock, turning on evolution for a certain amount of time. The original HHL algorithm suggests a weighted superposition of all states $\tau$ that minimizes errors in following steps in the algorithm. However, for our implementation, a uniform superposition already gives good results.

Our goal is to create a superposition of $A$ as a Hamiltonian applied for different durations. Since the eigenvalues are always situated on the complex unit circle, these differently evolved components in the superposition help reveal the eigenstructure. So we apply the conditional Hamiltonian evolution $\sum_{\tau=0}^{T-1}\left|\tau\right\rangle\left\langle\tau\right|\otimes e^{i A\tau t_0/T}$ on $\left|\psi_0\right\rangle\otimes\left|b\right\rangle$. This operation evolves the state $\left|b\right\rangle$ according to the Hamiltonian $A$ for the time $\tau$ determined by the state $\left|\psi_0\right\rangle$. Given that in $\left|\psi_0\right\rangle$ we have a superposition of all possible time steps between $0$ and $T$, we will end up with a superposition of all possible evolutions, and a suitable choice of number of timesteps $T$ and total evolution time $t_0$ allow to encode binary representations of the eigenvalues.

As a final step, we apply an inverse Fourier transformation that writes the phases (that, recall, encode the eigenvalues of $A$) into new registers.

The total circuit for phase estimation is the following:

![Quantum phase estimation in the quantum matrix inversion algorithm](figures/qpe_for_hhl.svg)


In our $2\times 2$ case, the circuit is massively simplified. Given that the matrix $A$ has eigenvalues that are powers of $2$, we can choose $T=4$, $t_0=2\pi$ to obtain exact results with just two controlled evolutions.

In [ ]:
# Superposition
hhl.append(cirq.H.on_each([q[1], q[2]]))
# Controlled-U0
hhl.append(cu3(-π/2, -π/2, π/2).on(q[2], q[3]))
hhl.append(cu1(3*π/4).on(q[2], q[3]))
hhl.append(cirq.CNOT.on(q[2], q[3]))
hhl.append(cu1(3*π/4).on(q[2], q[3]))
hhl.append(cirq.CNOT.on(q[2], q[3]))
# Controlled-U1
hhl.append(cirq.CNOT.on(q[1], q[3]))

We apply quantum inverse Fourier transformation to write the phase to a register:

In [ ]:
hhl.append(cirq.SWAP.on(q[1], q[2]))
hhl.append(cirq.H.on(q[2]))
hhl.append(cu1(-π/2).on(q[1], q[2]))
hhl.append(cirq.H.on(q[1]))

The state of the system after this decomposition is approximately $\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle \left|\lambda_{j}\right\rangle$, where $\left|b\right\rangle=\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle$ is the encoding of the vector $b$ in the eigenbasis of $A$. Now, there is an often overlooked step that performs bit operations on $\left|\lambda_{j}\right\rangle$ to actually invert it.

In our case, the inversion of the eigenvalues is easy. The eigenvalues of $A$ are $\lambda_1=2=10_2$ and $\lambda_2=1=01_2$, and their reciprocals are $\lambda_1^{-1}=1/2$ and $\lambda_2^{-1}=1$. Noting that $2\lambda_1^{-1}=01_2$ and $2\lambda_2^{-1}=10_2$, a swap gate is enough to obtain the state $\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle \left|2\lambda _{j}^{-1}\right\rangle$, that encodes the reciprocals of the eigenvalues.

In [ ]:
hhl.append(cirq.SWAP.on(q[1], q[2]))

# Conditional rotation of ancilla

Next, we perform a conditional rotation to encode the information of the reciprocals of the eigenvalues in the amplitudes of a state, on which we will later post-select. The state we would like to get is $\sum _{j{\mathop {=}}1}^{N}\beta _{j}\left|u_{j}\right\rangle\left|2\lambda _{j}^{-1}\right\rangle \left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\left|0\right\rangle+\frac{C}{\lambda_j}\left|1\right\rangle \right)$. This is achieved by controlled rotations in the same spirit of the conditional Hamiltonian evolution.

In [ ]:
hhl.append(cu3(0.392699, 0, 0).on(q[1], q[0]))   # Controlled-RY0
hhl.append(cu3(0.19634955, 0, 0).on(q[2], q[0])) # Controlled-RY1

# Uncomputing the eigenvalue register

A necessary step when performing quantum computations is to uncompute all operations except those that store the information that we want to obtain from the algorithm in the final registers. We need to do this in case the registers are entangled, which would affect the results.

In our case, we must uncompute the phase estimation protocol. After the uncomputation, the state should be $\sum_{j=1}^N\beta_j\left|u_j\right\rangle\left|0\right\rangle\left(\sqrt{1-\frac{C^2}{\lambda_j^2}}\left|0\right\rangle+\frac{C}{\lambda_j}\left|1\right\rangle \right)$, so we can safely forget about the eigenvalue register.

In [ ]:
hhl.append(cirq.SWAP.on(q[1], q[2]))
hhl.append(cirq.H.on(q[1]))
hhl.append(cu1(π/2).on(q[1], q[2])) # Inverse(Dagger(Controlled-S))
hhl.append(cirq.H.on(q[2]))
hhl.append(cirq.SWAP.on(q[2], q[1]))
# Inverse(Controlled-U1)
hhl.append(cirq.CNOT.on(q[1], q[3]))
# Inverse(Controlled-U0)
hhl.append(cirq.CNOT.on(q[2], q[3]))
hhl.append(cu1(-3*π/4)(q[2], q[3]))
hhl.append(cirq.CNOT.on(q[2], q[3]))
hhl.append(cu1(-3*π/4).on(q[2], q[3]))
hhl.append(cu3(-π/2, π/2, -π/2).on(q[2], q[3]))
# End of Inverse(Controlled-U0)
hhl.append(cirq.H.on_each([q[1], q[2]]))

# Rejection sampling on the ancilla register and a swap test

The state $\left|x\right\rangle=A^{-1}\left|b\right\rangle\propto\sum_j \beta_j\lambda_j^{-1}\left|u_j\right\rangle$ that contains information about the solution to $Ax=b$ is that obtained when measuring $1$ on the ancilla state. We perform the post-selection by projecting onto the desired $\left|1\right\rangle$. To check that the solution is the expected one, we prepare the correct output state manually to perform a swap test with the outcome.

In [ ]:
# Target state preparation
hhl.append(cirq.Rz(-π).on(q[4]))
hhl.append(u1(π).on(q[4]))
hhl.append(cirq.H.on(q[4]))
hhl.append(cirq.Ry(-0.9311623288419387).on(q[4]))
hhl.append(cirq.Rz(π).on(q[4]))
# Swap test
hhl.append(cirq.H.on(q[5]))
hhl.append(cirq.CNOT.on(q[4], q[3]))
hhl.append(cirq.CCX.on(q[5], q[3], q[4]))
hhl.append(cirq.CNOT.on(q[4], q[3]))
hhl.append(cirq.H.on(q[5]))

hhl.append(cirq.measure(*[q[0], q[5]], key='result'))

*Note: it is a good exercise to check that the right result is given by the state $\left|x\right\rangle=0.949\left|0\right\rangle + 0.314\left|1\right\rangle$, which is the state we prepare above.*

There are two measurements performed, one of the ancilla register (for doing the post-selection) and another one that gives the result of the swap test. To calculate success probabilities, let us define some helper functions.

In [ ]:
def get_psuccess(counts):
    '''Compute the success probability of the HHL protocol from the statistics

    :return: (float) The success probability.
    '''
    try:
        succ_rotation_fail_swap = counts['11']
    except KeyError:
        succ_rotation_fail_swap = 0
    try:
        succ_rotation_succ_swap = counts['01']
    except KeyError:
        succ_rotation_succ_swap = 0
    succ_rotation = succ_rotation_succ_swap + succ_rotation_fail_swap
    try:
        prob_swap_test_success = succ_rotation_succ_swap / succ_rotation
    except ZeroDivisionError:
        prob_swap_test_success = 0
    return prob_swap_test_success

Finally we run the circuit on the simulator:

In [ ]:
simulator = cirq.Simulator()
result = simulator.run(hhl, repetitions=100)
fold_func = lambda ms: ''.join(np.flip(ms, 0).astype(int).astype(str))
counts = result.histogram(key='result', fold_func=fold_func)
print(get_psuccess(counts))

Running on the actual QPU would yield a much poorer result due to imprecisions in the applications of the gates and noise caused by the environment.

# References
[1] J. Pan, Y. Cao, X. Yao, Z. Li, C. Ju, H. Chen, X. Peng, S. Kais, and J. Du. (2014). [Experimental realization of quantum algorithm for solving linear systems of equations](https://arxiv.org/abs/1302.1946). *Physical Review Letters* 89:022313. <a id='1'></a>
